<a href="https://colab.research.google.com/github/yeseul106/UROP_Software-Fault-Prediction/blob/main/UROP_NN_KC2_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import numpy as np
import pandas as pd

In [24]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
# csv 파일 읽어오기
CM1 = pd.read_csv('/content/drive/MyDrive/UROP/Dataset/KC2.csv', encoding='cp949', header=None)
print(CM1.shape)
CM1.head(5)

(522, 22)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,1.30,1.30,2,2,2,2,1.2,1.2,1.2,1.2,1.4,no
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1,1,1,1,1.0,1.0,1.0,1.0,1.0,yes
2,415.0,59.0,50.0,51.0,1159.0,8411.31,0.01,103.53,81.24,870848.58,2.80,48380.48,359,35,9,10,47.0,106.0,692.0,467.0,106.0,yes
3,230.0,33.0,10.0,16.0,575.0,3732.82,0.03,39.82,93.74,148644.06,1.24,8258.00,174,15,34,5,23.0,67.0,343.0,232.0,65.0,yes
4,175.0,26.0,12.0,13.0,500.0,3123.96,0.03,29.48,105.96,92103.07,1.04,5116.84,142,7,19,4,18.0,58.0,310.0,190.0,51.0,yes


In [26]:
# NA 값이 있는지 확인

CM1.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
dtype: int64

In [27]:
# 입력 데이터
input_data = CM1.loc[:,:20].to_numpy()
print("input_data shape : ",input_data.shape)

input_data shape :  (522, 21)


In [28]:
input_data[0]

array([1.1, 1.4, 1.4, 1.4, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 2. ,
       2. , 2. , 2. , 1.2, 1.2, 1.2, 1.2, 1.4])

In [29]:
# 정답 레이블
y_label = CM1.loc[:,21].to_numpy()
print("y_label shape : ",y_label.shape)

y_label shape :  (522,)


In [30]:
y_label[0]

'no'

In [31]:
for i in range(0, len(y_label)):
  if y_label[i]=='no':
    y_label[i]=False
  else:
    y_label[i]=True


In [32]:
y_label[0]

False

In [33]:
# 레이블 벡터화 
y_label = np.asarray(y_label).astype('float32')

# **정규화 (Normalization)**

In [34]:
from sklearn import preprocessing

# 정규화 0-1로 하기
data_normalizer = preprocessing.MinMaxScaler() # 데이터를 0~1 범위로 점철되게 하는 함수 call
data_normalized = data_normalizer.fit_transform(input_data) # 데이터를 0~1 범위로 점철되게 함수 수행
data_normalized.shape

(522, 21)

In [35]:
data_normalized[0]

array([7.84929356e-05, 2.23463687e-03, 3.22580645e-03, 2.81690141e-03,
       7.53579503e-05, 3.84449775e-05, 6.50000000e-01, 1.25567468e-02,
       3.13207729e-03, 6.05359676e-07, 1.15350488e-01, 8.49412217e-06,
       1.80668473e-03, 4.54545455e-02, 1.65289256e-02, 1.81818182e-01,
       4.34782609e-03, 3.69230769e-03, 8.10372771e-05, 7.93126239e-04,
       1.11111111e-03])

# **TrainData와 TestData의 분리**

In [36]:
from sklearn.model_selection import StratifiedKFold
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping

# 5개의 파일로 쪼갬
n_fold = 5
seed = 0
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

#빈 accuracy 배열
accuracy = []
loss = []

# **모델 설계**

# **실험 2**
- epoch : 200
- optimizer : adam 함수

In [37]:
from typing import Pattern
# 학습 자동 중단 설정
early_stopping_callback = EarlyStopping(monitor='loss', patience=200)

In [38]:
for train, test in skf.split(data_normalized, y_label):
  model = models.Sequential()
  model.add(layers.Dense(40, activation='relu', input_shape=(21,)))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(50, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(60, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['AUC'])
  hist = model.fit(data_normalized[train], y_label[train], epochs=600, batch_size=10, verbose=1, callbacks=[early_stopping_callback])
  k_accuracy = "%.4f"%(model.evaluate(data_normalized[test],y_label[test])[1])
  k_loss = "%.4f"%(model.evaluate(data_normalized[test],y_label[test])[0])
  accuracy.append(k_accuracy)
  loss.append(k_loss)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
42/42 [==============================] - 0s 4ms/step - loss: 0.2971 - auc: 0.9002
Epoch 507/600
42/42 [==============================] - 0s 3ms/step - loss: 0.3024 - auc: 0.8931
Epoch 508/600
42/42 [==============================] - 0s 3ms/step - loss: 0.2838 - auc: 0.9142
Epoch 509/600
42/42 [==============================] - 0s 3ms/step - loss: 0.3097 - auc: 0.8867
Epoch 510/600
42/42 [==============================] - 0s 4ms/step - loss: 0.2861 - auc: 0.8970
Epoch 511/600
42/42 [==============================] - 0s 4ms/step - loss: 0.2906 - auc: 0.9085
Epoch 512/600
42/42 [==============================] - 0s 3ms/step - loss: 0.3148 - auc: 0.8884
Epoch 513/600
42/42 [==============================] - 0s 4ms/step - loss: 0.2797 - auc: 0.9129
Epoch 514/600
42/42 [==============================] - 0s 3ms/step - loss: 0.2989 - auc: 0.8995
Epoch 515/600
42/42 [==============================] - 0s 3ms/step - loss: 0.2940 - auc: 0.9046
Epoch 516/600
42/4

In [39]:
print("\n %.f fold accuracy:" % n_fold, accuracy)
print("모든 fold의 평균 auc: ", (float(accuracy[0]) + float(accuracy[1]) + float(accuracy[2]) + float(accuracy[3]) + float(accuracy[4])) / 5)

print("\n %.f fold loss:" % n_fold, loss)
print("모든 fold의 평균 loss: ", (float(loss[0]) + float(loss[1]) + float(loss[2]) + float(loss[3]) + float(loss[4])) / 5)


 5 fold accuracy: ['0.8751', '0.8122', '0.8818', '0.6670', '0.8104']
모든 fold의 평균 auc:  0.8093

 5 fold loss: ['0.2995', '0.5757', '0.4297', '0.6967', '0.5607']
모든 fold의 평균 loss:  0.5124599999999999
